## Import packages

In [1]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [2]:
# Set path variables
path_to_adblocker = r"C:\Users\Matth\Desktop\3.12_0"
path_to_download_dir = r"C:\Users\Matth\Downloads"

In [3]:
# Number of parses to scrape (Current max is 100 or 1 page)
N_parses = 30

## Navigate to WCL

In [23]:
boss = "High Warlord Naj'entus"

In [14]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [15]:
boss_page_url, browser = src.load_top_N_scraper(path_to_adblocker, boss, boss_link_dict)

## Main code loop

In [36]:
def main():
    
    # First check to see if any of the rankings changed since last scraping and update the excel file
    for i in range(1, N_parses):
        rank, char_info, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)
        name = char_info.split("\n")[0]
        
        rank_changed = check_if_rank_changed(boss, rank, name, date)
        if rank_changed: 
            update_rank(boss, rank, name, date)
    
    for i in range(1, N_parses):
        rank, char_info, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)

        name = char_info.split("\n")[0]
        server_region = char_info.split("-")

        if len(server_region) == 1:
            server_region = char_info.split("\n")[1]
            server = server_region.split(" ")[0]
            region = server_region.split(" ")[1]

        else:
            server_region = char_info.split("-")[1]
            server = server_region.split(" ")[1]
            region = server_region.split(" ")[2]
        
        print(rank, name, server, region)
        
        if src.check_if_parse_already_recorded_top_N(boss, rank, name): 
                print("Parse already recorded. Skipping")
                continue
    
        link = browser.find_element_by_link_text(name)
        link.click()
        time.sleep(1)
    
        player_df = pd.DataFrame(pd.np.empty((0, 16)))
        player_df.columns = ["Rank", "Name", "Server", "Date", "Kill time", "nHealers", "Spriest?", "Innervate?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotations"]
    
        temp_url = browser.current_url
        
        boss_tanks = src.get_tanks(browser)
        nHealers = src.get_nHealers(browser)
    
        print(f"Tanks: {boss_tanks}")
        
        browser.get(temp_url)
        time.sleep(0.5)
        
        player_link = browser.find_element_by_link_text(name)
        player_link.click()
        time.sleep(0.5)
        
        # Scrape spell HPS
        LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, HPS)
        time.sleep(2)
                   
        # Scrape mana sources
        spriest = src.check_spriest(browser)
        innervate = src.check_innervate(browser)
        time.sleep(0.5)    
        
        # Download the cast-sequence CSV.
        src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "character_data/cast_sequence.csv")
        time.sleep(1)
        
        # Clean the csv
        df = src.clean_cast_sequence_csv()
        df = src.fix_cast_time(df)
        time.sleep(0.5)
        
        # Get the rotations
        rotations = src.calculate_rotations(df, boss, boss_tanks)
        print(f'Rotations: {rotations}')  

        to_append = [rank, name, server + " " + region, date, duration, str(nHealers), spriest, innervate, LB_uptime, HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotations]
        src.export_to_excel(boss, to_append, player_df, name, 'top_N_druids')
        
        os.remove(f"character_data/{boss.replace(' ', '')}_{name}.csv")
        os.remove(f"character_data/cast_sequence.csv")
        
        print('-------')
        browser.get(boss_page_url)
        time.sleep(1)
        
        
    print(f"{boss} scraping complete.")
    print("---------------")
    
    
if __name__ == "__main__":
    main()

Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.
Character rank did not change.


KeyboardInterrupt: 

In [11]:
browser.get(boss_page_url)

In [19]:
import openpyxl

In [35]:
for i in range(1, 50):
    rank_changed = False
    
    rank, char_info, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)
    name = char_info.split("\n")[0]
    
    rank_changed = check_if_rank_changed(boss, rank, name, date)
    if rank_changed: update_rank(boss, rank, name, date)

1 一杯马爹利 Mar 28
Character rank did not change.
2 氢硫化钠 Mar 21
Character rank did not change.
3 搞不得 Mar 17
Character rank did not change.
4 喵绿喵绿喵 Mar 19
Character rank did not change.
5 Дарнейм Feb 23
Character rank did not change.
6 芬巴巴之花 Mar 18
Character rank did not change.
7 黑夜现苍鹰 Mar 19
Character rank did not change.
8 德国进口德 Mar 24
Character rank did not change.
9 托尔德 Mar 18
Character rank did not change.
10 Day Mar 31
Character rank did not change.
11 小毛的毛毛 Mar 5
Character rank did not change.
12 尹冰冰 Mar 17
Character rank did not change.
13 飞奔的小毛 Mar 19
Character rank did not change.
14 Natures Mar 30
Character rank did not change.
15 Macdaddyswag Mar 3
Character rank did not change.
16 大鱼炖海棠 Mar 13
Character rank did not change.
17 大白兔生了 Mar 5
Character rank did not change.
18 丨照美冥丨 Mar 20
Character rank did not change.
19 白衣 Apr 2
20 Shallowmoon Mar 24
Character rank did not change.
21 廘盔 Mar 18
Character rank did not change.
22 馨香紫雨 Apr 2
23 Фунтович Feb 25
Rank changed!!
24 东加盏记

In [17]:
rank = 1
name = '一杯马爹利'
date = 'Mar 28'

In [37]:
def check_if_rank_changed(boss, rank, name, date):
    wb = openpyxl.load_workbook('character_data/top_N_druids.xlsx')

    boss = boss.replace(" ", "")

    ws = wb.get_sheet_by_name(boss)
    for row in ws.iter_rows():
        if [rank, name, date] == [row[0].value, row[1].value, row[3].value]:
            wb.save('character_data/top_N_druids.xlsx')
            return False
            
        elif [name, date] == [row[1].value, row[3].value] and rank != row[0].value:
            wb.save('character_data/top_N_druids.xlsx')
            return True
        
    wb.save('character_data/top_N_druids.xlsx')
    return False 

In [ ]:
def update_rank(boss, rank, name, date):
    wb = openpyxl.load_workbook('character_data/top_N_druids.xlsx')
    
    boss = boss.replace(" ", "")

    ws = wb.get_sheet_by_name(boss)
    for i, row in enumerate(ws.iter_rows()):
        if [name, date] == [row[1].value, row[3].value]:
            row[0].value = rank
        
    wb.save('character_data/top_N_druids.xlsx')